# Construction d'un neurone

On commence par construire un neurone simple, qu'on va ensuite tester. Je commance par l'exemple donné par [Miximum](https://www.miximum.fr/blog/introduction-au-deep-learning-1/) qui est basé sur des nombres aléatoires.

In [2]:
import numpy as np

Définition de la fonction Sigmoïde qui sera utilisée à l'intérieur de chaque neurone.

In [3]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

Pour le neurone, nous allons utiliser une classe définissant un objet neurone, qui aura un constructeur (nommé classiquement `__init__`) et une méthode d'activation. Cette méthode correspond au moment où le neurone reçoit des signaux des neurones qui lui sont connectés en amont. Le neurone pondère ces signaux (ici de manière aléatoire) et renvoie son signal qui sera donc capté par les neurones qui lui sont liés en aval. 

In [4]:
class Neuron:
    def __init__(self, input_size):
        """Initialisation des poids / biais avec des valeurs aléatoires."""
        self.weights = np.random.randn(input_size)
        self.bias = np.random.randn()
        
    def activation(self, X):
        """On suppose que X est de la taille passée dans le constructeur."""
        aggregation = np.sum(X * self.weights) + self.bias
        return sigmoid(aggregation)

Nous allons maintenant tester ce neurone. Nous construisons une premier couche appelée `Input` qui est vecteur de `n` valeurs dont les 5 premières sont à 1 et les autres à 0 (voir le fichier [01_numpy](../02_std_ext/01_numpy.ipynb)

In [25]:
n = 10
nb_zeros = 5
Uns = np.zeros(nb_zeros)
Zeros = np.ones(n - nb_zeros)
Input =  np.concatenate((Uns, Zeros))
Input

array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

Je construis maintenant mon neurone avec la classe définit précédement. Je me souviens que pour cela, le constructeur `__init__` demande de donner le nombre de neurones connectés en amont, donc ici mes `n` entrées.

In [26]:
mon_neurone = Neuron(n)

mon_neurone.activation(Input)


0.6987834466431921

# Construction d'un autre neurone



La distribution de poids de manière aléatoire ne me conviens pas et je veux donc pouvoir entrer des poids à mon neurone, à l'initialisation, mais aussi plus tard. Je rentrerai ces poids sous formes de vecteur et je n'ai plus besoin de rentrer une taille, car il s'agit simplement de la taille du vecteur des poids.

Je modifie donc mon programme de la manière suivante.

In [27]:
class Neuron2:
    def __init__(self, input_weights):
        """Initialisation des poids / biais avec des valeurs aléatoires."""
        self.weights = input_weights
        
    def activation(self, X):
        """On suppose que X est de la taille passée dans le constructeur."""
        aggregation = np.sum(X * self.weights)
        return sigmoid(aggregation)
    
    def update_weights(self, input_weights):
        """ mise à jour des poids """
        self.weights = input_weights

Pour utiliser mon neurone, il me faut toujours mon vecteur d'entrée `Input` mais aussi maintenant un vecteur contenant les poids à attacher à chaque entrée, je l'appelerai `Weight`. Je commence avec des poids égaux.

In [33]:
Weight = np.ones(n)
Weight

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [34]:
mon_neurone2 = Neuron2(Weight)

mon_neurone2.activation(Input)

0.9933071490757153

Si je bouleverse mes poids en mettant certains à zéro.

In [41]:
nb_zeros = 3
Zeros = np.zeros(nb_zeros)
Uns = np.ones(n - nb_zeros)
Weight =  np.concatenate((Uns, Zeros), axis=0)
Weight

array([1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

In [42]:
mon_neurone2.update_weights(Weight)

mon_neurone2.activation(Input)

0.8807970779778823

Pas de chance, je dégrade ici ma lecture.